In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy

import geopandas
import cartopy.io.shapereader as shpreader
import shapely

import seaborn as sns

In [2]:
degrees = 2.5
tdf = pd.read_csv(f'../data/study_da_6 - Temperature - upper_pred_{degrees}.csv')
tdf["da_var"] = "Temperature"
pdf = pd.read_csv(f'../data/study_da_6 - Precipitation - upper_pred_{degrees}.csv')
pdf["da_var"] = "Precipitation"

df = pd.concat([tdf,pdf])

df = df[pd.notna(df["gridcells"]) & df["gridcells"]>0]

df["da_trend_p"] = df["da_trend_cells"] / df["gridcells"]

df["da_trend_cat"] = None

df.loc[df['da_trend_p']==0,"da_trend_cat"] = "A. 0==DA"
df.loc[df['da_trend_p']>0,"da_trend_cat"] = "B. 0<DA<0.5"
df.loc[df['da_trend_p']>0.5,"da_trend_cat"] = "C. DA>0.5"

places =  pd.read_csv('../data/place_df.csv')

In [3]:
cat_df = pd.read_csv('../data/1_predicted_category_documents.csv')

df = df.merge(cat_df[["id","PY"]])